In [18]:
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from ClassificationReport import ClassificationMetrics

import ClassificationReport
import pandas as pd

In [19]:
# Load the data
X = np.load('data/processed/X.npy')
y = np.load('data/processed/y.npy', allow_pickle=True)

In [14]:
X.shape

(563, 57915)

In [12]:
pd.DataFrame(y).value_counts()

TNBC                      166
HER2+                     164
ER+                        85
ER+HER2+ LN metastasis     56
TNBC LN metastasis         54
ER+HER2+                   38
dtype: int64

In [20]:
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

# list all unique subtypes
unique_types = np.unique(y)
print("Unique types:", unique_types)

Shape of X: (563, 57915)
Shape of y: (563,)
Unique types: ['ER+' 'ER+HER2+' 'ER+HER2+ LN metastasis' 'HER2+' 'TNBC'
 'TNBC LN metastasis']


In [21]:
# Standardize the features
#scaler = StandardScaler()
#X_scaled = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Perform 5-fold cross-validation
cv_scores = cross_val_score(rf_classifier, X, y, cv=5)
print("Cross-Validation Scores:", cv_scores)
print("Mean CV Score:", cv_scores.mean())

# Train the classifier on the training data
rf_classifier.fit(X_train, y_train)

# Predict on the testing data
y_pred = rf_classifier.predict(X_test)

# Evaluate the classifier performance
#print(classification_report(y_test, y_pred, num_classes))
#classification_report_categorical(y_test, y_pred)
metrics = ClassificationMetrics(y_test, y_pred)
metrics.report()

#print("Accuracy on test data: ", accuracy_score(y_test, y_test))
#print("\nClassification Report:\n", classification_report(y_test, y_pred))
#print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Cross-Validation Scores: [0.7079646  0.84070796 0.84070796 0.83928571 0.61607143]
Mean CV Score: 0.7689475347661189
Class                         Precision Recall    F1-Score  
ER+                           1.00      1.00      1.00      
ER+HER2+                      0.75      0.50      0.60      
ER+HER2+ LN metastasis        0.78      0.70      0.74      
HER2+                         0.96      0.89      0.92      
TNBC                          0.76      0.97      0.85      
TNBC LN metastasis            0.83      0.45      0.59      

Overall Accuracy:             0.86
